# Plot and table creation for RTN-014

## Imports

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
from IPython.core.display import display, HTML
import sys
import os
from collections import namedtuple
from functools import partial
import sqlite3
import astropy
from astropy.time import Time
import astropy.coordinates
import astropy.units as u
import numpy as np
import pandas as pd
import scipy
import scipy.stats
import healpy
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from argparse import Namespace
import yaml
import logging
import numexpr
from logging import info

## Configuration

In [ ]:
logging.basicConfig(
    format='%(asctime)s %(message)s',
    level=logging.DEBUG)

mpl.rcParams['figure.figsize'] = (16, 5)
plt.style.use('ggplot')
np.random.seed(6563)

In [ ]:
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING) 

In [ ]:
BAND_COLOR_RUBIN = {
    'u': '#56b4e9',
    'g': '#008060',
    'r': '#ff4000',
    'i': '#850000',
    'z': '#6600cc',
    'y': '#000000'}

In [ ]:
# %aimport ddfpresched
# %aimport ddfpresched.m5
# %aimport ddfpresched.presched

In [ ]:
numexpr.set_num_threads(30)

## Read table of depth and other parameters by time

The data file read here can be created using the `python/ddfpresched/m5.py` script, using the configuration file in `etc/m5a.h5`.

Actually read the depth data into a `pandas.DataFrame`:

In [ ]:
%%time
short_survey = True
m5_fname = 'data/m5a.h5' if short_survey else 'data/m5.h5'
m5 = pd.read_hdf(m5_fname)

Take a look at what we've got:

In [ ]:
m5.describe().T

Set a useful index and sort by it.

To make debugging faster, cache it and just recopy it from the cache if the cell gets rerun.

In [ ]:
%%time
try:
    m5 = m5_checkpoint.copy()
except:
    m5 = m5.reset_index().set_index(['field_name', 'band', 'time'], drop=False).sort_index()
    m5_checkpoint = m5.copy()

When the field is below the horizon, it might have invalid values for airmass. Mark them as such.

In [ ]:
m5.loc[m5.field_airmass < 1.0, 'field_airmass'] = np.nan

Find the full moons, and mark them.

In [ ]:
moon_elongation = m5.sort_values('mjd').set_index('mjd').loc[:, 'moon_elongation'].drop_duplicates()
prev_moon_elongation = moon_elongation.shift(1)
next_moon_elongation = moon_elongation.shift(-1)
full_moon_mjds = moon_elongation.loc[np.logical_and(prev_moon_elongation < moon_elongation, next_moon_elongation < moon_elongation)].index.values
m5['full_moon'] = np.isin(m5.mjd, full_moon_mjds)
full_moon_nights = full_moon_mjds - m5.eval('mjd - night').median()
m5.query('full_moon').head()

Find where each field is closest to the moon. Note that this is independent of band, so we need only find the MJDs for one band.

In [ ]:
m5['moon_nearest'] = False
for field_name in m5.field_name.unique():
    these_m5 = m5.loc[(field_name, 'r')]
    moon_angle = these_m5.sort_values('mjd').set_index('mjd').loc[:, 'moon_angle'].drop_duplicates()
    prev_moon_angle = moon_angle.shift(1)
    next_moon_angle = moon_angle.shift(-1)
    near_moon_mjds = moon_angle.loc[np.logical_and(prev_moon_angle > moon_angle, next_moon_angle > moon_angle)].index.values
    m5.loc[field_name, 'moon_nearest'] = np.isin(m5.loc[field_name, 'mjd'], near_moon_mjds)
m5.query('moon_nearest').loc[('COSMOS', 'r'), 'moon_angle'].head()

## Minimum lunar angle with each field

In [ ]:
m5.groupby(level='field_name')['moon_angle'].min()

## Mark dates on which the moon is closest to the field

In [ ]:
field_m5 = m5.loc[('COSMOS', 'g')].groupby('night').min().reset_index().copy()
prev_moon_angle = field_m5.moon_angle.shift(1)
next_moon_angle = field_m5.moon_angle.shift(-1)
field_m5['moon_nearest'] = np.logical_and(prev_moon_angle > field_m5['moon_angle'],
                                              next_moon_angle > field_m5['moon_angle'])
field_m5.query('(moon_nearest or full_moon)').loc[:, ('time', 'mjd', 'night', 'field_airmass', 'full_moon', 'moon_nearest')]

# Depth and gap plots

In [ ]:
def make_dual_plot(in_m5, field_name=None, bands=None, gap_mags=[23, 23.5, 24], gap_band='g', show_legend=False, normalize=False, night_range=[200, 565], scheduled_nights=None, cadence_name=''):
    m5 = in_m5.query(f'(night>{night_range[0]}) and (night<{night_range[1]})')
    bands = reversed(('u', 'g', 'r', 'i', 'z', 'y')) if bands is None else bands
    fig, axes = plt.subplots(2, figsize=(12, 12), sharex=True)

    #for full_moon_night in m5.loc[(field_name, gap_band)].query('full_moon').night:
    for full_moon_night in full_moon_nights:
        for ax in axes:
            ax.axvline(x=full_moon_night, color='orange')


    field_m5 = m5.loc[(field_name, gap_band)].groupby('night').min().reset_index().copy()
    if True:
        prev_moon_angle = field_m5.moon_angle.shift(1)
        next_moon_angle = field_m5.moon_angle.shift(-1)
        field_m5['moon_nearest'] = np.logical_and(prev_moon_angle > field_m5['moon_angle'],
                                                  next_moon_angle > field_m5['moon_angle'])
    for nearest_moon_night in field_m5.query('moon_nearest').night:
        for ax in axes:
            ax.axvline(x=nearest_moon_night, color='yellow')

    for band in bands:
        these_nights = (m5
                        .loc[(field_name, band)]
                        .groupby('night')
                        .max()
                        .reindex(np.arange(np.max(m5.night))) # Drops downtime
                       )
        y = these_nights.m5 - these_nights.m5.max() if normalize else these_nights.m5
        axes[0].step(these_nights.index, y, label=band, color=BAND_COLOR_RUBIN[band], where='post')
        if band==gap_band:
            axes[1].step(these_nights.index, y, label=band, color=BAND_COLOR_RUBIN[band], where='post')
            
        if band==gap_band and scheduled_nights is not None:
            why_symbols = {'start': '*',
                           'cadence': '|',
                           'pregap': 5,
                           'postgap': 4,
                           'bridge': 'd'}
            in_time_scheduled_nights = scheduled_nights.loc[these_nights.mjd.min():these_nights.mjd.max()]
            for why_key in why_symbols:
                these_scheduled_nights = in_time_scheduled_nights.loc[in_time_scheduled_nights.why == why_key, 'night']
                axes[1].scatter(these_scheduled_nights, y.loc[these_scheduled_nights], color='magenta', marker=why_symbols[why_key], s=75)
        
    if len(gap_mags)>0:
        for gap_mag in gap_mags:
            best_mag = m5.loc[(field_name, gap_band), 'm5'].max() if normalize else 0
            good_nights = (m5
                           .loc[(field_name, gap_band)]
                           .groupby('night')
                           .max()
                           .reset_index()
                           .query(f'(m5 - {best_mag})>{gap_mag}')
                           .loc[:,['night']]
                           .copy())
            good_nights['next_night'] = good_nights.night.shift(-1)
            good_nights['next_gap'] = good_nights.next_night - good_nights.night
            gaps = good_nights.query('(next_gap>2) and (next_gap < 20)').copy()
            gaps = gaps.query('night > 5').copy()
            gaps['center_night'] = (gaps['night'] + gaps['next_night'])/2
            gap_mag_array = np.full(len(gaps), gap_mag)
            axes[1].hlines(y=gap_mag_array, xmin=gaps.night.values+1, xmax=gaps.next_night.values, color='black')
            axes[1].scatter(gaps.night.values+1, # Add 1 to draw line from the end of one night to the beginning of the next
                       gap_mag_array, color='black', marker='|')
            axes[1].scatter(gaps.next_night.values, gap_mag_array, color='black', marker='|')
            for _, gap in gaps.iterrows():
                axes[1].text(gap['center_night'],
                        gap_mag,
                        s=f'{gap["next_gap"].astype(int)}',
                        horizontalalignment='center',
                        verticalalignment='top')
    axes[0].set_title(f'Best limiting magnitudes by night for {field_name}')
    axes[1].set_title(f'{cadence_name} gaps in {field_name}')
    if show_legend:
        axes[0].legend()
    for ax in axes:
        nights_with_data = these_nights.dropna().query('m5>0').index.values
        ax.set_xlim(nights_with_data.min()-10, nights_with_data.max()+10)
        ax.set_ylabel('best single visit 5-sigma limiting mag. in night')
        ax.set_xlabel('night of survey')
        
    return fig, axes

In [ ]:
fig, axes = make_dual_plot(m5, 'COSMOS', ['u', 'g', 'r', 'i', 'z', 'y'], gap_mags=[-0.5, -1, -1.5], gap_band='g', normalize=True, show_legend=True, night_range=[300, 665])

In [ ]:
fig.savefig('figures/night_maglim_cosmos.png', dpi=600, bbox_inches="tight", pad_inches=0)
fig.savefig('figures/night_maglim_cosmos.pdf', bbox_inches="tight", pad_inches=0)

In [ ]:
fig, axes = make_dual_plot(m5, 'COSMOS', ['g', 'i', 'y'], gap_mags=[-0.5, -1, -1.5], gap_band='g', normalize=True, show_legend=True, night_range=[300, 665])

In [ ]:
fig, axes = make_dual_plot(m5, 'ECDFS', ['g', 'i', 'y'], gap_mags=[-0.5, -1, -1.5], gap_band='g', normalize=True, show_legend=True, night_range=[165, 540])

In [ ]:
fig, axes = make_dual_plot(m5, 'Elias S1', ['g', 'i', 'y'], gap_mags=[-0.5, -1, -1.5], gap_band='g', normalize=True, show_legend=True, night_range=[165, 540])

In [ ]:
fig, axes = make_dual_plot(m5, 'Euclid 1', ['g', 'i', 'y'], gap_mags=[-0.5, -1, -1.5], gap_band='g', normalize=True, show_legend=True, night_range=[200, 565])

In [ ]:
fig, axes = make_dual_plot(m5, 'XMM-LSS', ['g', 'i', 'y'], gap_mags=[-0.5, -1, -1.5], gap_band='g', normalize=True, show_legend=True, night_range=[200, 565])
fig.savefig('figures/night_maglim_xmmlss.png', dpi=600, bbox_inches="tight", pad_inches=0)
fig.savefig('figures/night_maglim_xmmlss.pdf', bbox_inches="tight", pad_inches=0)

# Schedule plots

In [ ]:
def make_schedule_plot(in_m5, schedule_fname, sequence_label, cadence_name, field_name='COSMOS',
                       gap_mags=[23, 23.5, 24], gap_band='g', show_legend=False, normalize=False, night_range=[300, 665],
                       breaking_gap=None,
                       breaking_maglim=None,
                       sequence_duration_min=40,
                       ylim=None):
    
    if schedule_fname is not None:
        scheduled_nights = (pd.read_csv(schedule_fname, sep="\t")
                            .set_index(['sequence', 'night_mjd'], drop=False)
                            .sort_index()
                            .loc[sequence_label])
        scheduled_nights['night'] = scheduled_nights['night_mjd'] - 59883
    else:
        scheduled_nights = None

    m5 = in_m5.query(f'(night>{night_range[0]}) and (night<{night_range[1]})')
        
    if breaking_gap is not None:
        if breaking_maglim is None:
            nongap_scheduled_nights = scheduled_nights.copy()
        else:
            scheduled_sequence_centers = (scheduled_nights
                                       .copy()
                                       .assign(center_mjd=scheduled_nights.mjd + sequence_duration_min/(24*60.0))
                                       .sort_values('center_mjd')
                                       .rename(columns={'night': 'sequence_night', 'mjd': 'sequence_mjd'})
            )
            scheduled_m5 = pd.merge_asof(
                scheduled_sequence_centers,
                m5.loc[(field_name, gap_band)].query('m5>10'),
                left_on='center_mjd',
                right_on='mjd',
                direction='nearest'
            )
            info(f"len(scheduled_m5): {len(scheduled_m5)}")
            nongap_night_mjd = scheduled_m5.query(f'm5 > {breaking_maglim}').night_mjd
            info(f"len(nongap_night_mjd): {len(nongap_night_mjd)}")
            nongap_scheduled_nights = scheduled_nights.loc[nongap_night_mjd]
            
        info(f"len(nongap_scheduled_nights): {len(nongap_scheduled_nights)}")
        nongap_scheduled_nights['subseq'] = (nongap_scheduled_nights['night_mjd'].diff() > breaking_gap).cumsum()
        scheduled_group_counts = (nongap_scheduled_nights
                                  .groupby('subseq')
                                  .count()['mjd']
                                  .reset_index()
                                  .rename({'mjd': 'count'}, axis='columns')
                                  .query('count>20')
                                 )
        scheduled_seasons = scheduled_group_counts['subseq']
        scheduled_nights = nongap_scheduled_nights.set_index('subseq').loc[scheduled_seasons].set_index('night_mjd', drop=False).sort_index()
        info(f'len(scheduled_nights): {len(scheduled_nights)}')

    # fig, axes = plt.subplots(figsize=(21, 6), sharex=True)
    fig, axes = plt.subplots(figsize=(12, 6), sharex=True)
    
    for full_moon_night in full_moon_nights:
        axes.axvline(x=full_moon_night, color='orange', linestyle=":")

    field_m5 = m5.loc[(field_name, gap_band)].groupby('night').min().reset_index().copy()
    prev_moon_angle = field_m5.moon_angle.shift(1)
    next_moon_angle = field_m5.moon_angle.shift(-1)
    field_m5['moon_nearest'] = np.logical_and(prev_moon_angle > field_m5['moon_angle'],
                                              next_moon_angle > field_m5['moon_angle'])
    for nearest_moon_night in field_m5.query('moon_nearest').night:
        axes.axvline(x=nearest_moon_night, color='yellow', linestyle=":")

    these_nights = (m5
                    .loc[(field_name, gap_band)]
                    .groupby('night')
                    .max()
                    .reindex(np.arange(np.max(m5.night))) # Drops downtime
                   )
    y = these_nights.m5 - these_nights.m5.max() if normalize else these_nights.m5
    axes.step(these_nights.index, y, label=gap_band, color=BAND_COLOR_RUBIN[gap_band], where='post')

    if scheduled_nights is not None:
        why_linestyles = {'start': '-.',
                          'cadence': '-',
                          'pregap': 'dashed',
                          'postgap': 'dashed',
                          'bridge': 'dotted'}
        in_time_scheduled_nights = scheduled_nights.loc[these_nights.mjd.min():these_nights.mjd.max()]
        ylim = axes.get_ylim() if ylim is None else ylim
        for why_key in why_linestyles:
            these_scheduled_nights = in_time_scheduled_nights.loc[in_time_scheduled_nights.why == why_key, 'night']
            if len(these_scheduled_nights)>0:
                axes.vlines(these_scheduled_nights.values, ylim[0], ylim[1], color='magenta', alpha=0.5, linestyle=why_linestyles[why_key])
                       
    # Adding the vlines expanded the y axis. Put it back
    axes.set_ylim(*ylim)
    axes.set_title(f'{cadence_name} cadence on {field_name}')

    nights_with_data = these_nights.dropna().query('m5>0').index.values
    axes.set_xlim(nights_with_data.min()-10, nights_with_data.max()+10)
    axes.set_ylabel('best single visit 5-sigma limiting mag. in night')
    
    night_xticks = axes.get_xticks()
    mjd_xticks = night_xticks + np.round(np.mean(m5.mjd - m5.night))
    axes.set_xticklabels("%d" % mjd for mjd in mjd_xticks)
    axes.set_xlabel('MJD')
        
    return fig, axes

In [ ]:
fig, ax = make_schedule_plot(m5,
                             'data/presched_2day.txt',
                             'COSMOS_g',
                             '2 day',
                             breaking_gap=8,
                             breaking_maglim=23)

In [ ]:
fig, ax = make_schedule_plot(m5,
                             'data/presched_2day_bridge_shallow.txt',
                             'COSMOS_g',
                             '2 day, bridged, shallow')

In [ ]:
fig, ax = make_schedule_plot(m5,
                             'data/presched_2day_bridge_shallow.txt',
                             'COSMOS_g',
                             '2 day, bridged, shallow',
                             breaking_gap=8)

In [ ]:
fig, ax = make_schedule_plot(m5,
                             'data/presched_2day_bridge.txt',
                             'COSMOS_g',
                             '2 day, bridged')

In [ ]:
fig, ax = make_schedule_plot(m5,
                             'data/presched_2day_bridge_deep.txt',
                             'COSMOS_g',
                             '2 day, bridged, deep')

In [ ]:
fig, ax = make_schedule_plot(m5,
                             'data/presched_2day.txt',
                             'COSMOS_g',
                             '2 day')

In [ ]:
fig, ax = make_schedule_plot(m5,
                             'data/presched_3day.txt',
                             'COSMOS_g',
                             '3 day')

In [ ]:
fig, ax = make_schedule_plot(m5,
                             'data/presched_2day.txt',
                             'XMM-LSS_g',
                             '2 day',
                             field_name='XMM-LSS',
                             night_range=[220, 475])

In [ ]:
fig, ax = make_schedule_plot(m5,
                             'data/presched_3day.txt',
                             'XMM-LSS_g',
                             '3 day',
                             field_name='XMM-LSS',
                             night_range=[220, 475])

# Rise and Set Plots

In [ ]:
m5['moon_airmass'] = 1.0/np.cos(np.radians(90-m5.moon_alt))
m5['field_alt'] = 90 - np.degrees(np.arccos(1.0/m5['field_airmass']))
m5['datetime'] = pd.to_datetime(m5['mjd']+2400000.5, unit='D', utc=True, origin='julian').dt.tz_convert('America/Santiago')

In [ ]:
m5[['night', 'full_moon']].reset_index(drop=True).query('full_moon').sort_values('night').drop_duplicates()

In [ ]:
def plot_alt_on_night(night,
                      m5,
                      field_names=['XMM-LSS'],
                      field_colors={'COSMOS': 'black', 'Elias S1': 'green', 'XMM-LSS': 'blue', 'ECDFS': 'red', 'Euclid 1': 'magenta'},
                      band='g',
                      fig=None,
                      ax=None):
    night_m5 = m5.query(f'night=={night}').sort_values('mjd')
    solar_midnight_mjd = night_m5[night_m5.sun_alt == night_m5.sun_alt.min()].mjd[0]
    night_m5['dmjd'] = (night_m5['mjd'] - solar_midnight_mjd)*24
    x_column = 'dmjd'
    
    if ax is None:
        fig, ax = plt.subplots()
    
    linestyle = '-'
    for field_name in field_names:
        field_table = night_m5.loc[(field_name, band)]
        field_table.plot(x_column, 'field_alt', label=field_name, color=field_colors[field_name], linestyle=linestyle, ax=ax)
        linestyle = ":"

    
    night_m5.plot(x_column, 'moon_alt', label='moon', ax=ax, color='orange')
    #night_m5.plot(x_column, 'sun_alt', label='sun', ax=ax, color='yellow')
    #ax.set_ylim(0, 90)
    #fig.autofmt_xdate()
    #ax.xaxis.set_major_formatter(mpl.dates.DateFormatter('%H:%M'))
    ax.set_xlabel('Hours after midnight')
    return ax


In [ ]:
def plot_alt_m5s(center_night,
             m5,
             night_shift=3,
             field_names=['XMM-LSS'],
             bands=None):
    bands = ('u', 'g', 'r', 'i', 'z', 'y') if bands is None else bands
    fig, axes = plt.subplots(6, 4, figsize=(12, 12), sharex=True)
    
    lunation_title = {0: 'early in season',
                      1: 'middle of season',
                      2: 'late in season'}
    night_title = {3: 'dark',
                   0: 'before full',
                   1: 'full',
                   2: 'after full'}
    
    # 59 is two synodic months
    for lunation_idx, full_night in enumerate([center_night-59, center_night, center_night+59]):
        for night_idx, obs_night in enumerate([full_night-night_shift, full_night, full_night+night_shift, full_night+14]):
            alt_ax = axes[2*lunation_idx, night_idx]
            plot_alt_on_night(obs_night, m5, field_names, ax=alt_ax, fig=fig)
            ax = axes[2*lunation_idx+1, night_idx]
            linestyle = '-'
            for field_name in field_names:
                for band in bands:
                    field_m5 = m5.loc[(field_name, band)].query(f'night=={obs_night}').sort_values('mjd')
                    solar_midnight_mjd = field_m5[field_m5.sun_alt == field_m5.sun_alt.min()].mjd[0]
                    field_m5['dmjd'] = (field_m5['mjd'] - solar_midnight_mjd)*24
                    field_m5.plot('dmjd', 'm5', label=band, color=BAND_COLOR_RUBIN[band], linestyle=linestyle, ax=ax)
                linestyle = ":"
            if alt_ax != axes[0,0]:
                alt_ax.get_legend().remove()
            if ax != axes[1,0]:
                ax.get_legend().remove()
            alt_ax.set_title(lunation_title[lunation_idx] + ', ' + night_title[night_idx])
            ax.set_xlim(-5, 5)
            ax.set_ylim(20.5, 25)
            alt_ax.set_ylim(0, 90)
             
    for column in range(4):
        axes[5, column].set_xlabel('Hours after midnight')
    
    for row in range(0, 6, 2):
        axes[row, 0].set_ylabel(r'Alt (deg)')
        axes[row+1, 0].set_ylabel(r'5-$\sigma$ mag. lim.')
    
    fig.tight_layout()
    return fig, axes

#plot_alt_m5s(481, m5, 4, field_names=['COSMOS'])
fig, axes = plot_alt_m5s(363, m5, 3, field_names=['XMM-LSS'])
fig.savefig('figures/m5_alt_xmmlss.png', dpi=600, bbox_inches="tight", pad_inches=0)
fig.savefig('figures/m5_alt_xmmlss.pdf', bbox_inches="tight", pad_inches=0)

# Sample simulations

## Get the simulation results

In [ ]:
%%bash
cd data
wget -nv https://lsst.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs_technical/ddf_ahead_dec/ddf_pre_fn0_v1.7_10yrs.db
wget -nv https://lsst.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs_technical/ddf_ahead_dec/ddf_pre_fn1_v1.7_10yrs.db
wget -nv https://lsst.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs_technical/ddf_ahead_dec/ddf_pre_fn2_v1.7_10yrs.db
wget -nv https://lsst.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs_technical/ddf_ahead_dec/ddf_pre_fn3_v1.7_10yrs.db
wget -nv https://lsst.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs_technical/ddf_ahead_dec/ddf_pre_fn4_v1.7_10yrs.db
wget -nv https://lsst.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs1.7/baseline/baseline_nexp1_v1.7_10yrs.db
wget -nv https://lsst.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs1.7/baseline/baseline_nexp2_v1.7_10yrs.db
wget -nv https://lsst.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs1.5/DDFs/agnddf_v1.5_10yrs.db
wget -nv https://lsst.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs1.5/DDFs/descddf_v1.5_10yrs.db
cd -

fn0 2day

fn1 2day bridge deep

fn2 3day

fn3 2day bridge 

fn4 2day bridge shallow 

In [ ]:
def find_field_hpix(nside, field_radius_deg, fld):
    hpxs = healpy.query_disc(nside, healpy.ang2vec(fld['ra'], fld['decl'], lonlat=True), np.radians(field_radius_deg))
    field_name = fld.name
    df = pd.DataFrame({'hpix': hpxs})
    df['field_name'] = field_name
    return df

In [ ]:
def load_ddf_visits(fname, include_wide=True, nside=512):
    logging.info("Reading %s", fname)
    ddf_fields = pd.DataFrame([{'ra': 9.45, 'decl': -44.0, 'field_name': 'Elias S1'},
                               {'ra': 35.708333, 'decl': -4-45/60., 'field_name': 'XMM-LSS'},
                               {'ra': 53.125, 'decl': -28.-6/60., 'field_name': 'ECDFS'},
                               {'ra': 150.1, 'decl': 2.+10./60.+55/3600., 'field_name': 'COSMOS'},
                               {'ra': 58.97, 'decl': -49.28, 'field_name': 'Euclid 1'},
                               {'ra': 63.6, 'decl': -47.60, 'field_name': 'Euclid 2'}
                              ]).set_index('field_name')

    with sqlite3.connect(fname) as con:
        visits = pd.read_sql_query('SELECT * FROM SummaryAllProps', con)
        
    field_radius_deg = np.sqrt(9.62/np.pi)
    ddf_hpix = (ddf_fields
                .groupby(level=0)
                .apply(lambda g: find_field_hpix(nside, field_radius_deg, g.iloc[0]))
                .set_index('hpix'))['field_name']
    
    visits['hpix'] = healpy.ang2pix(nside, visits['fieldRA'], visits['fieldDec'], lonlat=True)
    visits['field_name'] = visits['hpix'].map(ddf_hpix)
    visits['psnight'] = np.floor(0.2+visits['observationStartMJD'] - m5.mjd.min() + m5.night.min())
    
    visits['HA'] = ((180 + visits['observationStartLST'] - visits['fieldRA']) % 360) - 180
    visits['sunHA'] = ((180 + visits['observationStartLST'] - np.degrees(visits['sunRA'])) % 360) - 180
    
    if include_wide:
        ddf_visits = visits[np.logical_or(visits.field_name.notnull(), visits.note.str.startswith('DD'))].copy()
    else:
        ddf_visits = visits[visits.note.str.startswith('DD')].copy()
        
    logging.info("Finished Reading %s", fname)
    return ddf_visits

In [ ]:
def overplot_sim_on_schedule(visits, m5, field_name, band, schedule_fname, **kwargs):
    logging.info("Plotting %s in %s", field_name, band)
    these_visits = visits.query(f'field_name == "{field_name}" and filter=="{band}"')
    fig, ax = make_schedule_plot(m5,
                                 schedule_fname,
                                 field_name=field_name,
                                 gap_band=band,
                                 # ylim=(these_visits.fiveSigmaDepth.min(), these_visits.fiveSigmaDepth.max()),
                                 ylim=(21.5, 25.5),
                                 **kwargs
                                )
    xlabel = ax.get_xlabel()
    ylabel = ax.get_ylabel()
    these_visits.groupby('psnight').mean().reset_index().plot.scatter('psnight', 'fiveSigmaDepth', color='blue', marker='o', ax=ax)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    logging.info("Finished plotting %s in %s", field_name, band)
    return fig, ax

In [ ]:
def plot_sim_and_schedule(sim_fname, m5, field_name, band, schedule_fname, **kwargs):
    visits = load_ddf_visits(sim_fname)
    fig, ax = overplot_sim_on_schedule(visits, m5, field_name, band, schedule_fname, **kwargs)
    return fig, ax

In [ ]:
fig, ax = plot_sim_and_schedule('data/ddf_pre_fn0_v1.7_10yrs.db',
                                m5,
                                'COSMOS',
                                'g',
                                None, #'data/presched_2day.txt',
                                sequence_label='COSMOS_g',
                                cadence_name='2 day',
                                #night_range=[220, 475]
                               )
                                

In [ ]:
fig, ax = plot_sim_and_schedule('data/ddf_pre_fn0_v1.7_10yrs.db',
                                m5,
                                'XMM-LSS',
                                'g',
                                None, # 'data/presched_2day.txt',
                                sequence_label='XMM-LSS_g',
                                cadence_name='2 day',
                                night_range=[220, 475]
                               )
fig.savefig('figures/ddf_pre_fn0_v1.7_10yrs_xmmlss.png', dpi=600, bbox_inches="tight", pad_inches=0)
fig.savefig('figures/ddf_pre_fn0_v1_7_10yrs_xmmlss.pdf', bbox_inches="tight", pad_inches=0)

In [ ]:
fig, ax = plot_sim_and_schedule('data/ddf_pre_fn2_v1.7_10yrs.db',
                                m5,
                                'XMM-LSS',
                                'g',
                                None, #'data/presched_3day.txt',
                                sequence_label='XMM-LSS_g',
                                cadence_name='3 day',
                                night_range=[220, 475]
                               )
fig.savefig('figures/ddf_pre_fn2_v1.7_10yrs_xmmlss.png', dpi=600, bbox_inches="tight", pad_inches=0)
fig.savefig('figures/ddf_pre_fn2_v1_7_10yrs_xmmlss.pdf', bbox_inches="tight", pad_inches=0)

In [ ]:
fig, ax = plot_sim_and_schedule('data/baseline_nexp2_v1.7_10yrs.db',
                                m5,
                                'XMM-LSS',
                                'g',
                                None,
                                sequence_label='XMM-LSS_g',
                                cadence_name='baseline',
                                night_range=[220, 475]
                               )
fig.savefig('figures/baseline_nexp2_v1.7_10yrs_xmmlss.png', dpi=600, bbox_inches="tight", pad_inches=0)
fig.savefig('figures/baseline_nexp2_v1_7_10yrs_xmmlss.pdf', dpi=600, bbox_inches="tight", pad_inches=0)

In [ ]:
fig, ax = plot_sim_and_schedule('data/baseline_nexp2_v1.7_10yrs.db',
                                m5,
                                'COSMOS',
                                'g',
                                None,
                                sequence_label='COSMOS_g',
                                cadence_name='baseline',
                                # night_range=[220, 475]
                               )

In [ ]:
fig, ax = plot_sim_and_schedule('data/baseline_nexp2_v1.7_10yrs.db',
                                m5,
                                'ECDFS',
                                'g',
                                None,
                                sequence_label='ECDFS_g',
                                cadence_name='baseline',
                                night_range=[220, 475]
                               )

In [ ]:
fig, ax = plot_sim_and_schedule('data/agnddf_v1.5_10yrs.db',
                                m5,
                                'XMM-LSS',
                                'g',
                                None,
                                sequence_label='XMM-LSS_g',
                                cadence_name='AGN DDF',
                                night_range=[220, 475]
                               )
fig.savefig('figures/agnddf_v1.5_10yrs_xmmlss.png', dpi=600, bbox_inches="tight", pad_inches=0)
fig.savefig('figures/agnddf_v1_5_10yrs_xmmlss.pdf', bbox_inches="tight", pad_inches=0)

In [ ]:
fig, ax = plot_sim_and_schedule('data/descddf_v1.5_10yrs.db',
                                m5,
                                'XMM-LSS',
                                'g',
                                None,
                                sequence_label='XMM-LSS_g',
                                cadence_name='DESC DDF',
                                night_range=[220, 475]
                               )
fig.savefig('figures/desc_v1.5_10yrs_xmmlss.png', dpi=600, bbox_inches="tight", pad_inches=0)
fig.savefig('figures/desc_v1_5_10yrs_xmmlss.pdf', bbox_inches="tight", pad_inches=0)

# Gaps

In [ ]:
sim_fname = 'data/ddf_pre_fn0_v1.7_10yrs.db'
field_name = 'XMM-LSS'
band = 'g'
gap_bins = np.arange(0.5, 20.5, 1)
depths = (0, 22.5, 23, 23.5, 24)

In [ ]:
def make_gap_histogram_by_limit(sim_fname, field_name, band, gap_bins=np.arange(0.5, 20.5, 1), depths=(0, 22.5, 23, 23.5, 24)):
    visits = load_ddf_visits(sim_fname)

    these_visits = visits.query(f'field_name == "{field_name}" and filter=="{band}"')
    these_visits.head()

    night_visit_depths = these_visits.groupby('psnight').mean()[['observationStartMJD', 'fiveSigmaDepth']].rename(columns={'observationStartMJD': 'mjd'})

    fig, ax = plt.subplots(figsize=(12, 6), sharex=True)
    ax.hist([night_visit_depths.query(f'fiveSigmaDepth > {depth}').mjd.diff() for depth in depths],
                 bins=gap_bins,
                 label=[f'm>{depth}' for depth in depths])
    ax.set_xticks(np.arange(0, 20))
    ax.legend()
    return fig, ax

In [ ]:
make_gap_histogram_by_limit(
    sim_fname='data/ddf_pre_fn0_v1.7_10yrs.db',
    field_name='XMM-LSS',
    band='g'
)

In [ ]:
make_gap_histogram_by_limit(
    sim_fname='data/ddf_pre_fn1_v1.7_10yrs.db',
    field_name='XMM-LSS',
    band='g'
)

In [ ]:
make_gap_histogram_by_limit(
    sim_fname='data/ddf_pre_fn1_v1.7_10yrs.db',
    field_name='XMM-LSS',
    band='g'
)

In [ ]:
make_gap_histogram_by_limit(
    sim_fname='data/ddf_pre_fn2_v1.7_10yrs.db',
    field_name='XMM-LSS',
    band='g'
)

In [ ]:
make_gap_histogram_by_limit(
    sim_fname='data/ddf_pre_fn4_v1.7_10yrs.db',
    field_name='XMM-LSS',
    band='g'
)

In [ ]:
make_gap_histogram_by_limit(
    sim_fname='data/agnddf_v1.5_10yrs.db',
    field_name='XMM-LSS',
    band='g'
)

In [ ]:
make_gap_histogram_by_limit(
    sim_fname='data/descddf_v1.5_10yrs.db',
    field_name='XMM-LSS',
    band='g'
)

In [ ]:
make_gap_histogram_by_limit(
    sim_fname='data/baseline_nexp2_v1.7_10yrs.db',
    field_name='XMM-LSS',
    band='g'
)

## Histogram gaps by sim

In [ ]:
field_name = 'XMM-LSS'
#field_name = 'COSMOS'
band = 'g'
sim_fnames = ('data/baseline_nexp2_v1.7_10yrs.db',
              'data/descddf_v1.5_10yrs.db',
              'data/agnddf_v1.5_10yrs.db',
              'data/ddf_pre_fn0_v1.7_10yrs.db',
              'data/ddf_pre_fn2_v1.7_10yrs.db',
              'data/ddf_pre_fn3_v1.7_10yrs.db')
sim_colors = ('black', 'red', 'green', 'steelblue', 'skyblue', 'blue')

In [ ]:
try:
    sim_visits = cached_sim_visits.copy()
except:
    sim_visits = {sim_fname: load_ddf_visits(sim_fname).query(f'field_name == "{field_name}" and filter=="{band}"') for sim_fname in sim_fnames}
    cached_sim_visits = sim_visits.copy()

In [ ]:
def make_gap_histogram_by_sim(sim_visits, depth=0, gap_bins=np.arange(0.5, 14.5, 1), inset_bins=np.arange(0.5, 14.5, 1), colors=sim_colors):
    gaps = {sim_fname: these_visits
                       .groupby('psnight')
                       .mean()[['observationStartMJD', 'fiveSigmaDepth']]
                       .rename(columns={'observationStartMJD': 'mjd'})
                       .query(f'fiveSigmaDepth > {depth}')
                       .mjd.diff()
            for sim_fname, these_visits
            in sim_visits.items()}
                        
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.hist(gaps.values(),
            bins=gap_bins,
            label=tuple(gaps.keys()),
            color=colors
           )
    ax.set_xticks(np.arange(0, int(np.floor(gap_bins[-1]))+1))
    ax.legend()
    
    if depth>0:
        ax.set_title(f"m < {depth}")
    
    inset_ax = ax.inset_axes([0.3, 0.25, 0.7, 0.5])
    inset_ax.hist(gaps.values(), bins=gap_bins, label=tuple(gaps.keys()), color=colors)
    inset_ax.set_xlim(6.5, inset_bins[-1])
    inset_ax.set_ylim(0, 20)
    inset_ax.set_yticks(np.arange(0, 20, 4))
    
    return fig, ax

In [ ]:
gap_bins = np.arange(0.5, 10.5, 1)
gap_bins[-1] = 30
inset_bins = gap_bins

In [ ]:
fig, ax = make_gap_histogram_by_sim(sim_visits)
ax.set_title("Gaps between visits, all depths")
fig.savefig('figures/mag000_sim_gaps_xmmlss.png', dpi=600, bbox_inches="tight", pad_inches=0)
fig.savefig('figures/mag000_sim_gaps_xmmlss.pdf', bbox_inches="tight", pad_inches=0)

In [ ]:
fig, ax = make_gap_histogram_by_sim(sim_visits, depth=22.5)
ax.set_title("Gaps between visits, $m_{5\sigma,g}<22.5$")
fig.savefig('figures/mag225_sim_gaps_xmmlss.png', dpi=600, bbox_inches="tight", pad_inches=0)
fig.savefig('figures/mag225_sim_gaps_xmmlss.pdf', dpi=600, bbox_inches="tight", pad_inches=0)

In [ ]:
fig, ax = make_gap_histogram_by_sim(sim_visits, depth=23)
ax.set_title("Gaps between visits, $m_{5\sigma,g}<23$")
fig.savefig('figures/mag230_sim_gaps_xmmlss.png', dpi=600, bbox_inches="tight", pad_inches=0)
fig.savefig('figures/mag230_sim_gaps_xmmlss.pdf', bbox_inches="tight", pad_inches=0)

In [ ]:
fig, ax = make_gap_histogram_by_sim(sim_visits, depth=23.5)
ax.set_title("Gaps between visits, $m_{5\sigma,g}<23.5$")
fig.savefig('figures/mag235_sim_gaps_xmmlss.png', dpi=600, bbox_inches="tight", pad_inches=0)
fig.savefig('figures/mag235_sim_gaps_xmmlss.pdf', bbox_inches="tight", pad_inches=0)

## Tabulate gaps, season lengths, long gaps

In [ ]:
sim_visits.keys()

In [ ]:
long_gap_break = 7.5
season_break_gap = 30
depth = 20
sim_fname = 'data/ddf_pre_fn0_v1.7_10yrs.db'
#sim_fname = 'descddf_v1.5_10yrs.db'
#sim_fname = 'baseline_nexp2_v1.7_10yrs.db'
these_visits = (sim_visits[sim_fname]
                .query(f'fiveSigmaDepth > {depth}')
                .groupby('psnight')
                .mean()[['observationStartMJD', 'fiveSigmaDepth']]
                .rename(columns={'observationStartMJD': 'mjd'})
               )
these_visits['gap'] = these_visits['mjd'].diff()
these_visits['long_gap'] = these_visits['gap'] > long_gap_break
these_visits['interseason_gap'] = these_visits['gap'] > season_break_gap
these_visits['season'] = these_visits['interseason_gap'].apply(lambda x: 1 if x else 0).cumsum()
these_visits['sim_fname'] = sim_fname
these_visits.set_index('season', drop=False, inplace=True)
these_visits.loc[1].set_index('mjd', drop=False)

In [ ]:
ax = these_visits.plot.scatter('mjd', 'fiveSigmaDepth')
ax.set_xlim(60113, 60333)

In [ ]:
def compute_season_stats(df):
    count = len(df)
    season_start = df['mjd'].min()
    season_end = df['mjd'].max()
    season_length = season_end - season_start
    mean_gap = season_length/count
    try:
        max_gap = np.nanmax(df['gap'].values[1:])
    except:
        max_gap = np.nan
    long_gaps = len(df[df['long_gap']])
    
    # If the first gap is the interseason gap, do not count it
    first_long_gap = df['long_gap'].values[0]
    if (not np.isnan(first_long_gap)) and first_long_gap:
        long_gaps = long_gaps - 1
        
    season_stats = pd.Series({'season_start': season_start,
                                 'season_end': season_end,
                                 'season_length': season_length,
                                 'num_visits': count,
                                 'mean_gap': mean_gap,
                                 'max_gap': max_gap,
                                 'long_gaps': long_gaps})
    return season_stats

In [ ]:
def compute_sim_stats(sim_visits, depth=0, season_break_gap=28, long_gap_break=7.5, min_season_length=28):
    sim_night_list = []
    for sim_fname, sim_values in sim_visits.items():
        these_visits = (sim_values
                        .query(f'fiveSigmaDepth > {depth}')
                        .groupby('psnight')
                        .mean()[['observationStartMJD', 'fiveSigmaDepth']]
                        .rename(columns={'observationStartMJD': 'mjd'})
                       )
        these_visits['gap'] = these_visits['mjd'].diff()
        these_visits['long_gap'] = these_visits['gap'] > long_gap_break
        these_visits['interseason_gap'] = these_visits['gap'] > season_break_gap
        these_visits['season'] = these_visits['interseason_gap'].apply(lambda x: 1 if x else 0).cumsum()
        these_visits['sim_fname'] = sim_fname
        sim_night_list.append(these_visits)
        
    season_stats = (pd.concat(sim_night_list)
                    .groupby(['sim_fname', 'season'])
                    .apply(compute_season_stats)
                    .reset_index(level='season')
                   )
    
    season_stats = (season_stats
                    .groupby('sim_fname')
                    .apply(lambda df: df.nlargest(11, 'season_length')
                           .reset_index()
                           .drop(columns=['sim_fname']))
                   )
    
    sim_stats = (season_stats
                 .query(f'season_length > {min_season_length}')
                 .groupby('sim_fname')
                 .agg({'season': 'count',
                       'num_visits': 'sum',
                       'mean_gap': 'mean',
                       'max_gap': 'mean',
                       'long_gaps': 'sum',
                       'season_length': 'mean'})
                )
    
    sim_stats['num_visits'] = sim_stats['num_visits'].astype(int)
    sim_stats['long_gaps'] = sim_stats['long_gaps'].astype(int)
    sim_stats['max_gap'] = np.round(sim_stats['max_gap']).astype(int)
    sim_stats['season_length'] = np.round(sim_stats['season_length']).astype(int)
    sim_stats['mean_gap'] = np.round(sim_stats['mean_gap'], 1)
    return sim_stats

In [ ]:
sim_stats = compute_sim_stats(sim_visits)
print(sim_stats.to_latex())
sim_stats

In [ ]:
compute_sim_stats(sim_visits, depth=23)

In [ ]:
compute_sim_stats(sim_visits, depth=23.5)

In [ ]:
compute_sim_stats(sim_visits, depth=24, min_season_length=70)

In [ ]:
sim_names = {
    'data/agnddf_v1.5_10yrs.db': 'agnddf_v1.5_10yrs',
    'data/descddf_v1.5_10yrs.db': 'descddf_v1.5_10yrs',
    'data/baseline_nexp2_v1.7_10yrs.db': 'baseline_nexp2_v1.7_10yrs',
    'data/ddf_pre_fn0_v1.7_10yrs.db': '2 day prescheduled',
    'data/ddf_pre_fn2_v1.7_10yrs.db': '3 day prescheduled',
    'data/ddf_pre_fn3_v1.7_10yrs.db': '2 day prescheduled, bridged'
}

In [ ]:
sim_order = ('baseline_nexp2_v1.7_10yrs',
             'agnddf_v1.5_10yrs',
             'descddf_v1.5_10yrs',
             '2 day prescheduled',
             '3 day prescheduled',
             '2 day prescheduled, bridged')

sim_order_key = lambda sims: sims.map(lambda s: sim_order.index(s))

In [ ]:
def multi_sim_stats(sim_visits, **kwargs):
    sim_stats_list = []
    for depth in (20, 23.0, 23.5, 24):
        these_sim_stats = compute_sim_stats(sim_visits, depth=depth, min_season_length=70, **kwargs)
        these_sim_stats['depth'] = depth
        these_sim_stats.reset_index(inplace=True)
        these_sim_stats['sim'] = these_sim_stats['sim_fname'].map(sim_names)
        these_sim_stats.set_index('sim_fname')
        these_sim_stats.sort_values(by='sim', key=sim_order_key, inplace=True)
        sim_stats_list.append(these_sim_stats)

    sim_stats = pd.concat(sim_stats_list).set_index(['sim'], drop=True)
    sim_stats.drop('2 day prescheduled, bridged', inplace=True)

    sim_stats = sim_stats.reset_index().set_index(['depth', 'sim'], drop=True)
    return sim_stats

In [ ]:
sim_stats = multi_sim_stats(sim_visits)
sim_stats

In [ ]:
print(sim_stats[['num_visits', 'mean_gap', 'max_gap', 'long_gaps', 'season_length']].to_latex())

In [ ]:
sim_stats = multi_sim_stats(sim_visits)

trunc_sim_stats = multi_sim_stats(sim_visits, season_break_gap=13).reset_index(level='depth')
trunc_sim_stats = trunc_sim_stats.loc[['2 day prescheduled', '3 day prescheduled']]
trunc_sim_stats = trunc_sim_stats.reset_index().rename(columns={'sim': 'sim_src'})
trunc_sim_stats['sim'] = trunc_sim_stats['sim_src'].str.replace('prescheduled', 'pres. trunc.')
trunc_sim_stats = trunc_sim_stats.set_index(['depth', 'sim']).drop(columns=['sim_src'])
trunc_sim_stats.sort_index()

sim_stats = pd.concat([sim_stats, trunc_sim_stats]).sort_index()
sim_stats